# ANALYSE AI RISKS WITH GISKARD
cf https://www.giskard.ai/

In [ ]:
%pip install giskard -U

In [ ]:
from dissidentia.domain.model_wrapper import DissidentModelWrapper
from dissidentia.infrastructure.dataset import load_dataset
from giskard import Dataset, Model, scan
import pandas as pd
import numpy as np

In [ ]:
X, label = load_dataset()
df_raw = pd.concat([X, label], axis=1)
TEXT_COLUMN = "text"
TARGET_COLUMN = "label"
LABEL_MAPPING = {0: 0, 1: 1}

In [ ]:
giskard_dataset = Dataset(
    df=df_raw,
    target=TARGET_COLUMN,  # Ground truth variable.
    name="Dissident detection",  # Optional.
)

In [ ]:
model = DissidentModelWrapper.load("BertTypeClassifier").model

In [ ]:
def prediction_function(df: pd.DataFrame) -> np.ndarray:
    output = model.predict_proba(df[TEXT_COLUMN])
    return output[:, 1]

In [ ]:
giskard_model = Model(
    model=prediction_function,  # A prediction function that encapsulates all the data pre-processing steps and that
    model_type="classification",  # Either regression, classification or text_generation.
    name="Dissident detection",  # Optional
    classification_labels=list(
        LABEL_MAPPING.values()
    ),  # Their order MUST be identical to the prediction_function's
    feature_names=[TEXT_COLUMN],  # Default: all columns of your dataset
)

In [ ]:
results = scan(giskard_model, giskard_dataset)

In [ ]:
display(results)

In [ ]:
# Save it to a file
results.to_html("giskard_scan_report.html")